## Import and Set

In [1]:
import sys 
sys.path.append('../')
import os
import pickle
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from datas import data
from set_train import *
from models.Resnet import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
stock_symbol, end_date, num_class, batch_size, init, fp16_training, num_epochs, lr = set_train()
trainloader, validloader, testloader, test_date, df = data(stock_symbol, num_class, end_date, batch_size, window=100)
torch.cuda.empty_cache()

100%|██████████| 2863/2863 [00:04<00:00, 605.32it/s]


x_train_len: 2543, valid_len: 160, test_len: 160


In [2]:
for x, y in trainloader:
    print(x.shape, x.device)

torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([160, 6, 100, 100]) cuda:0
torch.Size([143, 6, 100, 100]) cuda:0


## Init: Model, Criteria, Optimizer, Fp16, Load previous trained

In [3]:
if fp16_training:
    from accelerate import Accelerator
    accelerator = Accelerator()
    device = accelerator.device
    model = resnet50(num_class)
else:
    model = resnet50(num_class).to(device)
Model = model.model_type

# Check path
if os.path.exists(f'Temp//{Model}_{stock_symbol}_LastTrainInfo.pk'):
    if init:
        print("Init model")
        lr = lr
        last_epoch = 0
        min_val_loss = 10000
        loss_train = []
        loss_valid = []
    else:
        print('Load from last train epoch')
        model.load_state_dict(torch.load(f'Temp//{Model}_class{num_class}_{stock_symbol}_checkpoint_LastTrainModel.pt'))
        with open(f'Temp//{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk', 'rb') as f:
            last_train_info = pickle.load(f)
        with open(f'Temp//{Model}_class{num_class}_{stock_symbol}_TrainValHistLoss.pk', 'rb') as f:
            loss_train_val = pickle.load(f)            
        lr = last_train_info['lr']
        last_epoch = last_train_info['epoch']
        min_val_loss = last_train_info['min val loss']
        loss_train = loss_train_val['train']
        loss_valid = loss_train_val['valid']
else:
        print("Init model")
        lr = lr
        last_epoch = 0
        min_val_loss = 10000.0
        loss_train = []
        loss_valid = []
        
print(
    f'Last train epoch: {last_epoch}  '
    f'Last train lr: {lr}   '
    f'Min val loss: {min_val_loss}'
    )

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)        

if fp16_training:
    print('Accelerate Prepare')    
    model, optimizer, trainloader, validloader, scheduler = \
    accelerator.prepare(model, optimizer, trainloader, validloader, scheduler)
        
for name, param in model.named_parameters():
    print(f"Parameter '{name}' is on device: {param.device}")
    break

Init model
Last train epoch: 0  Last train lr: 1e-05   Min val loss: 10000.0
Accelerate Prepare
Parameter 'conv1.weight' is on device: cuda:0


## Train

In [ ]:
for epoch in range(last_epoch, num_epochs):
    # Training phase
    model.train()
    loss_train_e = 0
    for batch_x, batch_y in tqdm(trainloader):
        if not fp16_training:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward() if not fp16_training else accelerator.backward(loss)     
        optimizer.step()
        scheduler.step()
        loss_train_e += loss.item()
    loss_train_e /= len(trainloader)
    loss_train.append(loss_train_e)
    
    # Scheduler 
    if epoch > 200:
        scheduler.step()
    
    loss_valid_e = 0
    with torch.no_grad():
        model.eval()
        for batch_x_val, batch_y_val in tqdm(validloader):
            batch_x_val = batch_x_val.to(device)
            batch_y_val = batch_y_val.to(device)
            outputs_val = model(batch_x_val)
            loss = criterion(outputs_val, batch_y_val)
            loss_valid_e += loss.item()
        loss_valid_e /= len(validloader)
        loss_valid.append(loss_valid_e)            
        torch.save(model.state_dict(), f'Temp/{Model}_class{num_class}_{stock_symbol}_checkpoint_LastTrainModel.pt')
        if loss_valid_e < min_val_loss:
            min_val_loss = loss_valid_e
            print(f'New best model found in epoch {epoch} with val loss: {min_val_loss}')
            torch.save(model.state_dict(), f'Model_Result/{Model}_class{num_class}_{stock_symbol}_best_model.pt')            
        if epoch % 50 == 0:
            pass
            
    with open(f'Temp/{Model}_class{num_class}_{stock_symbol}_TrainValHistLoss.pk', 'wb') as f:
        pickle.dump({'train': loss_train, 'valid': loss_valid}, f)
    with open(f'Temp/{Model}_class{num_class}_{stock_symbol}_LastTrainInfo.pk', 'wb') as f:
        pickle.dump({'min val loss': min_val_loss, 'epoch': epoch, 'lr': optimizer.param_groups[0]['lr']}, f)
    print(
        f'Epoch [{epoch}/{num_epochs}]',
        f'Training Loss: {loss_train_e:.10f}',
        f'Valid Loss: {loss_valid_e:.10f}'
        )
